In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [4]:
data_train=pd.read_csv('train.csv')
data_test=pd.read_csv('test.csv')

data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [16]:
#relleno nulos
data_train['Age']=data_train['Age'].fillna(data_train['Age'].mean())
#mapeo male a 0 y female a 1
data_train['Sex'] = data_train['Sex'].map({'male': 0, 'female': 1})
#Separamos Features (X) y Target (y)
features=['Pclass','Sex','Age','SibSp','Parch','Fare']
x_raw = data_train[features].values 
y_raw = data_train['Survived'].values.reshape(-1, 1)
# NORMALIZACIÓN (CRÍTICO PARA REDES NEURONALES)
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_raw)

In [17]:
print("--- Data Engineering Completado ---")
print(f"Forma de X: {x_scaled.shape}") # Debería ser (891, 6)
print(f"Forma de y: {y_raw.shape}")     # Debería ser (891, 1)
print(f"Ejemplo de datos (fila 0): \n{x_scaled[0]}")

--- Data Engineering Completado ---
Forma de X: (891, 6)
Forma de y: (891, 1)
Ejemplo de datos (fila 0): 
[ 0.82737724 -0.73769513 -0.5924806   0.43279337 -0.47367361 -0.50244517]


1) Crea la clase TitanicDataset (Igual que hiciste con HouseDataset).

2) Instánciala con los nuevos datos.

3) Crea el DataLoader (batch_size=32, shuffle=True).

4) Define la arquitectura TitanicModel.

* Entrada: Ahora son 6 features (Pclass, Sex, Age, SibSp, Parch, Fare).

* Ocultas: Usa 2 capas ocultas (puedes probar con 32 neuronas cada una).

* Salida: 1 neurona (binaria).

* Activación: ReLU en las ocultas, nada en la salida.

In [ ]:
class TitanicDataset(Dataset):
    def __init__(self, features,targets):
        self.features=torch.tensor(data=features, dtype=torch.float32)
        self.targets=torch.tensor(data=targets, dtype=torch.float32)
    def __len__(self) -> int:
        return len(self.features)
    def __getitem__(self, index):
        # DEVOLVEMOS TUPLA: (Datos, Precio)
        return self.features[index], self.targets[index]
#instanciamos el dataset
titanic_dataset=TitanicDataset(x_scaled, y_raw)
#Crear el DataLoader
#batch_size=32: Entregará paquetes de 32 datos
#shuffle=True: Barajará los datos en cada época
loader = DataLoader(dataset=titanic_dataset, batch_size=32, shuffle=True)

In [19]:
class TitanicModel(nn.Module):
    def __init__(self):
        super().__init__()
        #primera de 3 a 32 ocultas. Entrada -> Oculta
        self.capa1=nn.Linear(in_features=6,out_features=32)
        self.activacion=nn.ReLU()
        #oculta -> oculta. segunda transformación (Profundidad): De 32 a 32
        self.capa2=nn.Linear(in_features=32,out_features=32)
        #capa Final (Compresión): De 32 a 1 solo precio
        self.salida=nn.Linear(in_features=32,out_features=1)
    
    def forward(self, x):
        """Debe pasar x a través de: capa1 -> activacion -> capa2 -> activacion -> salida."""
        x=self.capa1(x)
        x=self.activacion(x)
        x=self.capa2(x)
        x=self.activacion(x)
        x=self.salida(x)
        return x

In [27]:
modelo=TitanicModel()
#loss clasificacion binaria
loss_function=nn.BCEWithLogitsLoss()
#optimizador
optimizador=torch.optim.Adam(params=modelo.parameters(),lr=0.01)
#bucle
epochs=50

for epoch in range(epochs):
    loss_acumulada=0
    correctos_acumulados = 0
    total_muestras = 0

    for batch_x, batch_y in loader:      
        # 1. Forward
        prediccion = modelo(batch_x)
        # 2. Calcular Loss
        loss = loss_function(prediccion, batch_y)
        # 3. Limpiar gradientes previos
        optimizador.zero_grad()
        # 4. Backward (Calcular gradientes)
        loss.backward()
        # 5. Actualizar pesos
        optimizador.step()
        # Acumulamos el error para monitorear (opcional)
        loss_acumulada += loss.item()

        #calculo accuracy
        #convertir logits a probabilidades
        probs = torch.sigmoid(prediccion)
        #Convertir a 0 o 1 (Decisión)
        preds = (probs > 0.5).float()
        #Contar aciertos (True si son iguales, False si no)
        correctos_acumulados += (preds == batch_y).sum().item()
        total_muestras += batch_y.size(0) # Sumamos 32 (tamaño del batch)
    # Reporte por epoch
    promedio_loss = loss_acumulada / len(loader)
    accuracy_epoch = correctos_acumulados / total_muestras
    print(f"Époch {epoch+1} | Loss Promedio: {promedio_loss:.5f} | Accuracy: {accuracy_epoch*100:.2f}%")


Époch 1 | Loss Promedio: 0.50849 | Accuracy: 76.43%
Époch 2 | Loss Promedio: 0.43295 | Accuracy: 80.47%
Époch 3 | Loss Promedio: 0.42022 | Accuracy: 82.27%
Époch 4 | Loss Promedio: 0.41042 | Accuracy: 82.27%
Époch 5 | Loss Promedio: 0.40946 | Accuracy: 82.27%
Époch 6 | Loss Promedio: 0.40983 | Accuracy: 82.27%
Époch 7 | Loss Promedio: 0.40676 | Accuracy: 82.94%
Époch 8 | Loss Promedio: 0.41284 | Accuracy: 81.93%
Époch 9 | Loss Promedio: 0.41400 | Accuracy: 83.73%
Époch 10 | Loss Promedio: 0.39684 | Accuracy: 82.72%
Époch 11 | Loss Promedio: 0.40339 | Accuracy: 82.49%
Époch 12 | Loss Promedio: 0.39398 | Accuracy: 84.06%
Époch 13 | Loss Promedio: 0.38941 | Accuracy: 83.05%
Époch 14 | Loss Promedio: 0.38460 | Accuracy: 84.18%
Époch 15 | Loss Promedio: 0.38625 | Accuracy: 83.16%
Époch 16 | Loss Promedio: 0.38180 | Accuracy: 83.28%
Époch 17 | Loss Promedio: 0.39154 | Accuracy: 82.94%
Époch 18 | Loss Promedio: 0.38404 | Accuracy: 82.72%
Époch 19 | Loss Promedio: 0.38667 | Accuracy: 83.16%
Ép

Para cerrar este curso con broche de oro, vamos a hacer una inferencia divertida pero técnica. Vamos a preguntarles a tu modelo qué opina sobre los protagonistas de la película: Jack y Rose.

El problema de la Escala (Recordatorio Crítico): Tu modelo aprendió viendo datos como "Edad: 0.5" (normalizado), no "Edad: 20". Si le pasas los datos crudos de Jack, la red pensará que tiene una edad gigante y fallará. 👉 Regla: Debemos usar el mismo scaler que usaste en el entrenamiento para transformar a Jack y Rose.

Datos de los Personajes:
1) Jack Dawson:

    * Clase: 3 (Pobre)

    * Sexo: Male (0)

    * Edad: 20 años

    * Hermanos/Esposa (SibSp): 0

    * Padres/Hijos (Parch): 0

    * Tarifa: 7.25 (Barato)

2) Rose DeWitt Bukater:

    * Clase: 1 (Rica)

    * Sexo: Female (1)

    * Edad: 17 años

    * Hermanos/Esposa (SibSp): 0

    * Padres/Hijos (Parch): 1 (Su madre)

    * Tarifa: 100.00 (Caro, estimado)

In [28]:
# 1. Definimos los datos crudos [Pclass, Sex, Age, SibSp, Parch, Fare]
pasajeros = np.array([
    [3, 0, 20, 0, 0, 7.25],   # Jack
    [1, 1, 17, 0, 1, 100.00]  # Rose
])

# 2. ESCALADO (Vital: Usamos el scaler que ya entrenaste)
pasajeros_scaled = scaler.transform(pasajeros)

# 3. Convertir a Tensor
# RELLENA AQUÍ: Crea el tensor float32
pasajeros_tensor = torch.tensor(pasajeros_scaled, dtype=torch.float32)

# 4. Inferencia
modelo.eval() # Modo evaluación
with torch.no_grad():
    # RELLENA AQUÍ: Forward pass
    logits = modelo(pasajeros_tensor)
    
    # RELLENA AQUÍ: Sigmoide para obtener probabilidad
    probs = torch.sigmoid(logits)

# 5. Resultados
nombres = ["Jack", "Rose"]
for i, nombre in enumerate(nombres):
    p = probs[i].item()
    estado = "Sobrevive" if p > 0.5 else "Muere"
    print(f"{nombre}: {p*100:.2f}% de probabilidad -> {estado}")

Jack: 10.60% de probabilidad -> Muere
Rose: 98.29% de probabilidad -> Sobrevive
